# Connect to VARS database and pull down data

**Note** that in order for this to work, in addition to VARS_logon_info.txt and VARS.py, you will need the following driver file in your working directory: <span style="color:blue"> mssql-jdbc-8.2.2.jre8.jar </span>

For Lis Henderson

In [1]:
## Imports

import pandas as pd
import numpy as np

import re # for extracting logon info from text file

import jaydebeapi # for connecting to VARS db
import VARS # for connecting to VARS db

In [2]:
## Extract logon information from text file -- This uses a guest username and pw that will let you download data but not overwrite anything!

# Get list of each line in file
filename = 'VARS_logon_info.txt'
f = open(filename, 'r')
lines = f.readlines()
f.close()

# Function for extracting information from lines
def get_single_quoted_text(s):
    """ 
    Takes string s and returns any text in s that is between the first set of single quotes, removing whitespace. 
    
    Example:
    s = "What if there's more ' than one' sest of single' quotes?"
    get_single_quoted_text(s) --> 's more'
    
    """
    
    extracted_text = re.search('''(?<=')\s*[^']+?\s*(?=')''', s)
    return(extracted_text.group().strip())

# Assign logon info
dr = get_single_quoted_text(lines[2])
name = get_single_quoted_text(lines[3])
pw = get_single_quoted_text(lines[4])
un = get_single_quoted_text(lines[5])
url = get_single_quoted_text(lines[6])

In [3]:
## Build SQL query -- This query grabs select columns (see those listed under SELECT) from the annotations table for non-embargoed records from 1989 (see last two lines)

sql = """
        SELECT index_recorded_timestamp,
               observation_uuid,
               concept,
               observation_group,
               observer,
               image_url,
               depth_meters,
               latitude,
               longitude,
               oxygen_ml_per_l,
               psi,
               salinity,
               temperature_celsius,
               video_uri,
               video_sequence_name,
               chief_scientist
        FROM annotations a
        WHERE NOT EXISTS (
           SELECT DISTINCT observation_uuid
           FROM annotations b
           WHERE (
             (  -- Delete last 2 years of annotations
             index_recorded_timestamp > DATEADD([year], - 2, GETDATE()) OR
             index_recorded_timestamp IS NULL OR
             index_recorded_timestamp < CAST('1970-01-02' AS datetime)
             )
           OR ( -- Delete embargoes by dive
             dive_number IN ('Ventana 50', 'Ventana 217', 'Ventana 218', 'Ventana 248')
              )
           OR (
             dive_number IN ('Tiburon 1001', 'Tiburon 1029', 'Tiburon 1030', 'Tiburon 1031', 'Tiburon 1032', 'Tiburon 1033', 'Tiburon 1034')
             )
           OR ( -- Delete embargoes by selectedConcept
             concept IN (
                 'Aegina sp. 1',
                 'Ctenophora',
                 'Cydippida 2',
                 'Cydippida',
                 'Intacta',
                 'Llyria',
                 'Lyrocteis',
                 'Lyroctenidae',
                 'Mertensia',
                 'Mertensiidae sp. A',
                 'Mystery Mollusc',
                 'Mystery Mollusc',
                 'Physonectae sp. 1',
                 'Platyctenida sp. 1',
                 'Platyctenida',
                 'Thalassocalycida sp. 1',
                 'Thalassocalycida',
                 'Thliptodon sp. A',
                 'Tjalfiella tristoma',
                 'Tjalfiella',
                 'Tjalfiellidae',
                 'Tuscarantha braueri',
                 'Tuscarantha luciae',
                 'Tuscarantha',
                 'Tuscaretta globosa',
                 'Tuscaretta',
                 'Tuscaridium cygneum',
                 'Tuscaridium',
                 'Tuscarilla campanella',
                 'Tuscarilla nationalis',
                 'Tuscarilla similis',
                 'Tuscarilla',
                 'Tuscarora',
                 'Tuscaroridae'
                 )
            )
        ) AND a.observation_uuid = b.observation_uuid
    ) AND index_recorded_timestamp >= CAST('1989-01-01' AS datetime) 
      AND index_recorded_timestamp <= CAST('1989-12-31' AS datetime)
    """

I don't know if you're familiar with SQL. The VARS database is a Microsoft SQL Server database. Some other simpler queries:

Get all columns of all records in the annotations table from 2017.
```python
sql2 = """
    SELECT * 
    FROM annotations a
    WHERE index_recorded_timestamp >= CAST('2017-01-01' AS datetime)
        AND index_recorded_timestamp <= CAST('2017-12-31' AS datetime)
       """
```

Get time and species columns for all Dosidicus gigas sightings.
```python
sql3 = """
    SELECT index_recorded_timestamp
           concept
    FROM annotations a
    WHERE concept in ('Dosidicus', 
                      'Dosidicus gigas', 
                      'Humboldt squid')
       """
```

In [4]:
## Query the database

# Get connection
conn = VARS.get_db_conn(dr, url, un, pw, name)

# Submit query
data, col_names = VARS.get_data(conn, sql)

# Close connection
conn.close()

**Just FYI:** In my original notebook, I say that jaydebeapi was throwing an Attribute Error ("java.sql.Types has no attribute \_\_javaclass__") in the \_jdbc_connect_jpype function in jaydebeapi. Although I couldn't find the exact error online, it seemed related to another which was due to the newest (0.7.0) installation of JPype being buggy. I used:

```python
pip install --user JPype1==0.6.3 --force-reinstall
```

in the command line to install a previous version, and that fixed it.

In [7]:
## Check data is there

# Set column names
data.columns = col_names

print(data.shape)
data.head()

(44741, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,1989-08-18 18:07:37,D72E0E85-A1C9-491C-B3DF-C807584FF4C9,Salpida,ROV,unknown,None,256.140015,36.711683,-122.044252,None,96.0,34.150002,7.974,urn:tid:mbari.org:V0062-06,Ventana 0062,Bill Hamner
1,1989-08-18 17:52:14,6BB5A15E-92BC-4B8E-9E4D-B4A665408BD8,larvacean house,ROV,unknown,None,245.229996,36.711990,-122.044002,None,186.0,34.125999,8.114,urn:tid:mbari.org:V0062-06,Ventana 0062,Bill Hamner
2,1989-12-21 21:11:16,2500E9E8-B0CF-4358-BEF2-420FDF428237,Laminariales,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0110-04,Ventana 0110,Chris Harrold
3,1989-12-21 21:11:16,2500E9E8-B0CF-4358-BEF2-420FDF428237,Laminariales,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0110-04,Ventana 0110,Chris Harrold
4,1989-12-20 21:36:59,22DFA75B-5E8B-440B-BBA7-E28E97E09217,Stylasterias forreri,ROV,unknown,None,360.899994,36.716754,-121.989487,None,96.0,34.174000,7.038,urn:tid:mbari.org:V0109-11,Ventana 0109,Chuck Baxter


In [ ]:
## Save data, since it will be a lot faster to load rather than pulled data from the database every time

data.to_csv('data.csv', index=False, na_rep='NaN')